# Project 2: Customer‑Support Chatbot for an E-Commerce Store

Welcome! In this project, you'll build a **chatbot** that answers customer service questions about Everstorm Outfitters, an imaginary e-commerce store.

Run each cell in order. Feel free to modify them as you go to better understand each tool and search the web or look online for documentation.

## Learning Objectives  
* **Ingest & chunk** unstructured docs  
* **Embed** chunks and **index** with *FAISS*  
* **Retrieve** context and **craft prompts**  
* **Run** an open‑weight LLM locally with *Ollama*  
* **Build** a Retrieval-Augmented Generation (RAG) chain
* **Package** the chat loop in a minimal **Streamlit** web UI

## Roadmap  
We will build a RAG-based chatbot in **six** steps:

1. **Environment setup**
2. **Data preparation**  
   a. Load source documents  
   b. Chunk the text  
3. **Build a retriever**  
   a. Generate embeddings  
   b. Build a FAISS vector index  
4. **Build a generation engine**. Load the *Gemma3-1B* model through Ollama and run a sanity check.  
5. **Build a RAG**. Connect the system prompt, retriever, and LLM together. 
6. **(Optional) Streamlit UI**. Wrap everything in a simple web app so users can chat with the bot.


## 1 - Environment setup

We use conda to manage our project dependencies and ensure everyone has a consistent setup. Conda is an open-source package and environment manager that makes it easy to install libraries and switch between isolated environments. To learn more about conda, you can read: https://docs.conda.io/en/latest/

Create and activate a clean *conda* environment and install the required packages. If you don't have conda installed, visit https://www.anaconda.com/docs/getting-started/miniconda/main.


Open your terminal, navigate to the project folder where this notebook is located, and run the following commands.

```bash
conda env create -f environment.yml && conda activate rag-chatbot

# (Optional but recommended) Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=rag-chatbot --display-name "rag-chatbot"
```
Once this is done, you can select “rag-chatbot” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, solves all pinned dependencies, and builds an isolated environment named `rag-chatbot`.
> * When it reaches the file’s "pip:" section, Conda automatically invokes pip to install any remaining Python-only packages so the whole stack be available for the project.
> * Registering the kernel makes your new environment visible to Jupyter, so the notebook runs inside the same environment you just created.

In [4]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langchain]/7 [langgraph]


In [5]:
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-ollama
!pip install langchain-huggingface
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-ollama

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.0
    Uninstalling langchain-core-1.0.0:
      Successfully uninstalled langchain-core-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-text-splitters 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-ollama 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-community 0.4 re

Let's import required libraries and print a message if we're not **missing packages**.

In [4]:
# Import standard libraries for file handling and text processing
import os, pathlib, textwrap, glob

# Load documents from various sources (URLs, text files, PDFs)
from langchain_community.document_loaders import UnstructuredURLLoader, TextLoader, PyPDFLoader

# Split long texts into smaller, manageable chunks for embedding
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Vector store to store and retrieve embeddings efficiently using FAISS
from langchain_community.vectorstores import FAISS

# Generate text embeddings using OpenAI or Hugging Face models
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings

# Use local LLMs (e.g., via Ollama) for response generation
from langchain_ollama import ChatOllama

# Build a retrieval chain that combines a retriever, a prompt, and an LLM
# Note: We'll use RetrievalQA chain from langchain_classic
from langchain_classic.chains import RetrievalQA

# Create prompts for the RAG system
from langchain_core.prompts import PromptTemplate

print("✅ Libraries imported! You're good to go!")

✅ Libraries imported! You're good to go!


In [5]:
# Load all Everstorm PDF files using PyPDFLoader
import glob

# Find all PDF files matching the pattern
pdf_files = glob.glob("data/Everstorm_*.pdf")
print(f"Found {len(pdf_files)} PDF files:")
for pdf_file in pdf_files:
    print(f"  - {pdf_file}")

# Load all PDFs and collect pages in raw_docs
raw_docs = []

for pdf_file in pdf_files:
    print(f"\nLoading {pdf_file}...")
    try:
        # Create PyPDFLoader instance for this PDF
        loader = PyPDFLoader(pdf_file)
        # Load all pages from the PDF
        pages = loader.load()
        raw_docs.extend(pages)
        print(f"  ✓ Loaded {len(pages)} pages")
    except Exception as e:
        print(f"  ✗ Error loading {pdf_file}: {e}")

print(f"\n📚 Total documents loaded: {len(raw_docs)} pages")
print(f"📄 Sample document metadata:")
if raw_docs:
    print(f"  - Source: {raw_docs[0].metadata.get('source', 'N/A')}")
    print(f"  - Page: {raw_docs[0].metadata.get('page', 'N/A')}")
    print(f"  - Content preview: {raw_docs[0].page_content[:200]}...")


Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)


Found 8 PDF files:
  - data/Everstorm_Return_and_exchange_policy.pdf
  - data/Everstorm_Return_and_exchange_policy 2.pdf
  - data/Everstorm_Product_sizing_and_care_guide.pdf
  - data/Everstorm_Shipping_and_Delivery_Policy 2.pdf
  - data/Everstorm_Product_sizing_and_care_guide 2.pdf
  - data/Everstorm_Shipping_and_Delivery_Policy.pdf
  - data/Everstorm_Payment_refund_and_security.pdf
  - data/Everstorm_Payment_refund_and_security 2.pdf

Loading data/Everstorm_Return_and_exchange_policy.pdf...
  ✓ Loaded 2 pages

Loading data/Everstorm_Return_and_exchange_policy 2.pdf...
  ✗ Error loading data/Everstorm_Return_and_exchange_policy 2.pdf: Cannot read an empty file

Loading data/Everstorm_Product_sizing_and_care_guide.pdf...
  ✓ Loaded 2 pages

Loading data/Everstorm_Shipping_and_Delivery_Policy 2.pdf...
  ✗ Error loading data/Everstorm_Shipping_and_Delivery_Policy 2.pdf: Cannot read an empty file

Loading data/Everstorm_Product_sizing_and_care_guide 2.pdf...
  ✗ Error loading data/Everstor

## 2 - Data preparation
The goal of this step is to turn all reference documents into small chunks of text that a retriever can index and search. These documents typically come from:
* PDF files: local documents such as policies, user manuals, or guides.
* Web pages (HTML): online documentation, blog posts, or help articles.

In this step, we perform two actions:
* **Ingesting**: load every PDF and collect the raw text in a list named `raw_docs`.
* **Chunking**: split each document into small, overlapping chunks so later steps can match a user query to the most relevant passage.

### 2.1 - Ingest source documents

We can use different libraries to load and process PDFs. A quick web search will show several options, each with its own strengths. In this case, we’ll use PyPDFLoader from LangChain, which makes it easy to extract text from PDF files for downstream processing. To learn more about how to use it, refer to: https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/

Use **PyPDFLoader** to load every PDF whose filename matches `data/Everstorm_*.pdf` and collect all pages in a list called `raw_docs`. The content of these PDFs is synthetically generated for educational purposes.

In [7]:
!pip install pypdf

In [8]:
pdf_paths = glob.glob("data/Everstorm_*.pdf")
raw_docs = []

for pdf_path in pdf_paths:
    try:
        loader = PyPDFLoader(pdf_path)
        raw_docs.extend(loader.load())
        print(f"Loaded {len(raw_docs)} PDF pages from {len(pdf_paths)} files.")
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")
        continue


print(f"Loaded {len(raw_docs)} PDF pages from {len(pdf_paths)} files.")

Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)


Loaded 2 PDF pages from 8 files.
Error loading data/Everstorm_Return_and_exchange_policy 2.pdf: Cannot read an empty file
Loaded 4 PDF pages from 8 files.
Error loading data/Everstorm_Shipping_and_Delivery_Policy 2.pdf: Cannot read an empty file
Error loading data/Everstorm_Product_sizing_and_care_guide 2.pdf: Cannot read an empty file
Loaded 6 PDF pages from 8 files.
Loaded 8 PDF pages from 8 files.
Error loading data/Everstorm_Payment_refund_and_security 2.pdf: Cannot read an empty file
Loaded 8 PDF pages from 8 files.


### (Optional) 2.1 - Load web pages
You can also pull content straight from the web. Various libraries support reading and parsing web pages directly into text, which is useful for building custom knowledge bases. One example is **UnstructuredURLLoader** from LangChain, which can extract readable content from raw HTML pages and return them in a structured format. To learn more, see: https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.url.UnstructuredURLLoader.html

To practice, load each HTML page below and store the results in a list called `raw_docs`. We’ve included a few sample URLs, but you can replace them with any links you prefer.

For robustness, add an offline fallback in case a URL fails. In real projects, we typically cache fetched pages to disk, handle rate limits, and track fetch timestamps so content can be refreshed periodically without relying on live network calls during development. For this project, we don’t have offline HTML copies available, but you can still practice by loading any PDFs from the data/ folder using PyPDFLoader and appending them to raw_docs.

In [9]:
URLS = [
    # --- BigCommerce – shipping & refunds ---
    "https://developer.bigcommerce.com/docs/store-operations/shipping",
    "https://developer.bigcommerce.com/docs/store-operations/orders/refunds",
    # --- Stripe – disputes & chargebacks ---
    # "https://docs.stripe.com/disputes",
    # --- WooCommerce – REST API reference ---
    # "https://woocommerce.github.io/woocommerce-rest-api-docs/v3.html",
]

try:
    ########################
    #### Your code here (~2-3 lines of code) ####
    ########################
    print(f"Fetched {len(raw_docs)} documents from the web.")
except Exception as e:
    print("⚠️  Web fetch failed, using offline copies:", e)
    raw_docs = []
    ########################
    #### Your code here ####
    ########################
    print(f"Loaded {len(raw_docs)} offline documents.")

Fetched 8 documents from the web.


### 2.2 - Chunk the text

Long documents won’t work well directly with most LLMs. They can easily exceed the model’s context window, making it impossible for the model to read or reason over the full text at once. Even if they fit, processing long inputs can be inefficient and lead to weaker retrieval results.

To handle this, we split large documents into smaller, overlapping chunks. Several libraries can help with text splitting, each designed to preserve structure or balance chunk size. A popular choice is `RecursiveCharacterTextSplitter` from LangChain, which splits text intelligently while keeping paragraph or sentence boundaries intact. To familiarize youself with the library, visit: https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

In this project, we’ll split each document into chunks of roughly 300 tokens with a 30-token overlap using `RecursiveCharacterTextSplitter`. This overlap helps maintain continuity across chunks while ensuring each piece stays small enough for embedding and retrieval.

In [10]:
chunks = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(raw_docs)
print(f"✅ {len(chunks)} chunks ready for embedding")

✅ 42 chunks ready for embedding


## 3 -Build a retriever

A *retriever* lets the RAG pipeline efficiently look up small, relevant pieces of context at query‑time. This step has two parts:
1. **Load a model to generate embeddings**: convert each text chunk from the reference documents into a fixed‑length vector that captures its semantic meaning.  
2. **Build vector database**: store these embeddings in a vector database.


### 3.1 - Load a model to generate embeddings

The goal of this step is to convert each document chunk into a numerical vector (an embedding) that captures its semantic meaning. These embeddings allow our retriever to find and compare similar pieces of text efficiently.

There are models trained specifically for this purpose, called embedding models. One popular example is OpenAI’s `text-embedding-3-small`, which produces high-quality embeddings that work well for retrieval and semantic search.

If you prefer running everything locally, you can use smaller open-source models such as `gte-small` (77 M parameters). These local models load quickly, don’t require internet access, and are ideal for experimentation or environments without API access. However, they’re typically less powerful than hosted models.

Alternatively, you can connect to an API service to access stronger models like OpenAI’s. These require setting an API key (for example, OPENAI_API_KEY) in your environment. OpenAI allows you to create a free account and sometimes offers limited trial credits for new users, but ongoing access requires a billing setup. 

In this exercise, we’ll stick to the smaller gte-small model for simplicity and reproducibility. We'll use our imported `SentenceTransformerEmbeddings` library to load the model and use it to embed queries. To learn more about lagnchain's embedding support, visit: https://python.langchain.com/docs/integrations/text_embedding/

In [11]:
!pip install sentence-transformers

In [13]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embedding_vector = []

# Embed the sentence "Hello world! and store it in an embedding_vector.
embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Extract text content from Document objects
texts = [chunk.page_content for chunk in chunks]
embedding_vector = embedder.embed_documents(texts)

print(len(embedding_vector))

42


### 3.2 - Build a vector database

Once we have embeddings, we need a way to store and search them efficiently. A simple list wouldn’t scale well, especially when we have thousands of chunks and need to quickly find the most relevant ones.

To solve this, we use **FAISS**, an open-source similarity search library developed by Meta. FAISS is optimized for fast nearest-neighbor search in high-dimensional spaces, making it ideal for tasks like semantic retrieval and recommendation. It’s strongly encouraged to visit their quickstart guide to understand how FAISS works and how to use it effectively: https://github.com/facebookresearch/faiss/wiki/getting-started

In this step, we’ll feed all our document embeddings into FAISS, which builds an in-memory vector index. This index allows us to efficiently query for the *k* most similar chunks to any given question.

During inference, we’ll use this index to retrieve the top-k relevant chunks and pass them to the LLM as context, enabling it to answer questions grounded in our documents.



In [15]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached faiss_cpu-1.12.0-cp313-cp313-macosx_14_0_arm64.whl.metadata (5.1 kB)
Using cached faiss_cpu-1.12.0-cp313-cp313-macosx_14_0_arm64.whl (3.4 MB)


In [16]:
# Expected steps:
    # 1. Build the FAISS index from the list of document chunks and their embeddings.
    # 2. Create a retriever object with a suitable k value (e.g., 8).
    # 3. Save the vector store locally (e.g., under "faiss_index").
    # 4. Print a short confirmation showing how many embeddings were stored.
vectordb = FAISS.from_documents(chunks, embedder)
retriever = vectordb.as_retriever(search_kwargs={"k": 8})
vectordb.save_local("faiss_index")

print("✅ Vector store with", vectordb.index.ntotal, "embeddings")

✅ Vector store with 42 embeddings


## 4 - Build the generation engine
At the core of any RAG system lies an **LLM**. The retriever finds relevant information, and the LLM uses that information to generate coherent, context-aware responses.

In this project, we’ll use **Gemma 3* (1B), a small but capable open-weight model, and run it entirely on your local machine using Ollama. This means you won’t need API keys or internet access to generate responses once the model is downloaded.

**What is Ollama?**

Ollama is a lightweight runtime for managing and serving open-weight LLMs locally. It provides:
* A simple REST API running at localhost:11434, so your code can interact with the model via standard HTTP calls.
* A model registry and command-line tool** to pull, run, and manage models easily.
* Support for a wide variety of models (e.g., Gemma, Llama, Mistral, Phi, etc.), making it ideal for experimentation.

To learn more about Ollama, visit: https://github.com/ollama/ollama. You can browse all supported models and their sizes here: https://ollama.com/library


### 4.1 - Install `ollama` and serve `gemma3`

Follow these steps to set up Ollama and start the model server:

**1 - Install**
```bash
# macOS (Homebrew)
brew install ollama
# Linux
curl -fsSL https://ollama.com/install.sh | sh
```

If you’re on Windows, install using the official installer from https://ollama.com/download.

**2 - Start the Ollama server (keep this terminal open)**
```bash
ollama serve
```
This command launches a local server at http://localhost:11434, which will stay running in the background.


**3 - Pull the Gemma mode (or the model of your choice) in a new terminal**
```bash
ollama pull gemma3:1b
```

This downloads the 1B version of Gemma 3, a compact model suitable for running on most modern laptops. Once downloaded, Ollama will automatically handle model loading and caching.


After this setup, your system is ready to generate responses locally using the Gemma model through the Ollama API.


### 4.2 - Test an LLM with a random prompt (Sanity check)


In [18]:
# Expected steps:
    # 1. Initialize the model (for example, gemma3:1b) with a low temperature such as 0.1 for more factual outputs.
    # 2. Use llm.invoke() with a short test prompt and print the response to verify that the model runs successfully.
llm = ChatOllama(model="gemma3:1b", temperature=0.1)
print(llm.invoke("What is 2+2?"))

content='2 + 2 = 4\n' additional_kwargs={} response_metadata={'model': 'gemma3:1b', 'created_at': '2025-10-18T17:22:07.283383Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1660938708, 'load_duration': 1094872833, 'prompt_eval_count': 16, 'prompt_eval_duration': 407545750, 'eval_count': 9, 'eval_duration': 135666457, 'model_name': 'gemma3:1b', 'model_provider': 'ollama'} id='lc_run--930f38dc-c561-4886-9d07-511cee3b81d7-0' usage_metadata={'input_tokens': 16, 'output_tokens': 9, 'total_tokens': 25}


## Build a RAG

### 5.1 - Define a system prompt

At this stage, we need to tell the model how to behave when generating answers. The **system prompt** acts as the model’s rulebook. It should clearly instruct the model to answer only using the retrieved context and to admit when it doesn’t know the answer. This helps prevent hallucination and keeps the responses grounded in the provided documents.

In general, a good RAG prompt emphasizes three things: stay within context, stay factual, and stay concise. This is important because RAG works by grounding the LLM in retrieved text. If the prompt is vague, the model may invent details. A precise system prompt reduces hallucinations and keeps answers aligned with your corpus.

In [19]:
SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
"""

### 5.2 Create a RAG chain
Now that we have a retriever, a prompt, and a language model, we can connect them into a single RAG pipeline. The retriever finds the most relevant chunks from our vector index, the prompt injects those chunks into the system message, and the LLM uses that context to produce the final answer. (retriever → prompt → model)

This connection is handled through LangChain’s `ConversationalRetrievalChain`, which combines retrieval and generation. To familiarize yourself with the library, visit: https://python.langchain.com/api_reference/langchain/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html

In [22]:
# Expected steps:
    # 1. Create a PromptTemplate that uses the SYSTEM_TEMPLATE you defined earlier, with input variables for "context" and "question".
    # 2. Initialize your LLM using Ollama with the gemma3:1b model and a low temperature (e.g., 0.1) for reliable, grounded responses.
    # 3. Build a ConversationalRetrievalChain by combining the LLM, the retriever, and your custom prompt and name it "chain".

prompt = PromptTemplate(template=SYSTEM_TEMPLATE, input_variables=["context", "question"])
llm = ChatOllama(model="gemma3:1b", temperature=0.1)
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", chain_type_kwargs={"prompt": prompt})

When you ask a question, the retriever pulls the top few relevant text chunks, the model reads them through the system prompt, and then it generates an answer based on that context.

This structure makes the system transparent and easy to debug. You can inspect what text was retrieved, tune parameters like k, and experiment with different prompts to see how they affect the output quality.


### 5.3 - Validate the RAG chain

We run a few questions to make sure everything behaves as expecte. Experiment by adding you own questions.

In [ ]:
test_questions = [
    "If I'm not happy with my purchase, what is your refund policy and how do I start a return?",
    "How long will delivery take for a standard order, and where can I track my package once it ships?",
    "What's the quickest way to contact your support team, and what are your operating hours?",
]

# Create a simple template without the source issue
simple_template = """
You are a Customer Support Chatbot. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with "I'm not sure from the docs."

CONTEXT:
{context}

USER:
{question}
"""

chat_history = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"Question {i}: {question}")
    print(f"{'='*60}")

    # Get relevant documents
    docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Create the prompt with simple template
    formatted_prompt = simple_template.format(context=context, question=question)

    # Get response from LLM
    response = llm.invoke(formatted_prompt)
    answer = response.content

    print(f"Answer: {answer}")
    print(f"{'='*60}")

    chat_history.append((question, answer))


Question 1: If I'm not happy with my purchase, what is your refund policy and how do I start a return?
Answer: Okay, here’s the information from the context regarding our refund policy:

Our refund policy is as follows:

*   **No return unless defective.**
*   **Refund Timeline:**
    *   Warehouse receipt → inspection ≤ 3 business days
    *   Refund initiates.
    *   Stripe/Apple Pay: 5-7 banking days
    *   PayPal Instant: 10 business days
    *   Shop-Pay: Up to 10 business days
*   **Defect & Warranty Claims (12):**  These are handled within 12 days of the return.
*   **Return Window Exceptions:**
    *   Holiday gift purchases made between November 1st and December 31st: return period extended to January 31st.
    *   “Final Sale” and custom-embroidered items have a return period of 31 January.
*   **Where’s my Klarna statement?** Klarna emails the payment schedule shortly after order confirmation. If not received, log in at [link to Klarna login].

**Do you charge sales tax?*

### 6 - Build the Streamlit UI (optional)

The goal here is to create a tiny demo so you can interact with your RAG system. The focus is not on UI design. We will build a very small interface only to demonstrate the end-to-end flow.

There are many ways to make a UI. Some frameworks are powerful but take longer to set up, while others are simple and good for quick experiments. Streamlit is a common choice for fast prototyping because it lets you make a usable interface with only a few lines of Python. If you want to learn the basics, see the Streamlit Quickstart: https://docs.streamlit.io/deploy/streamlit-community-cloud/get-started/quickstart

This step is optional. If it is not useful for your work, you can skip it. We will also complete this part together during the live session.

In this cell, we write a minimal **`app.py`** that starts a simple chat UI and calls your RAG chain.

In [32]:
!pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached pyarrow-21.0.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 16.5 MB/s  0:00:00eta 0:00:01
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
Using cached pyarrow-21.0.0-cp313-cp313-macosx_12_0_arm64.whl (31.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [strea

In [33]:
import streamlit as st
import os
import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

# Page configuration
st.set_page_config(page_title="Everstorm Customer Support", page_icon="🤖")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Load and process documents (this could be cached in production)
@st.cache_resource
def load_documents():
    # Load PDFs
    pdf_files = glob.glob("data/Everstorm_*.pdf")
    raw_docs = []

    for pdf_file in pdf_files:
        try:
            loader = PyPDFLoader(pdf_file)
            pages = loader.load()
            raw_docs.extend(pages)
        except Exception as e:
            st.error(f"Error loading {pdf_file}: {e}")

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
    chunks = text_splitter.split_documents(raw_docs)

    # Create embeddings and vector store
    embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = FAISS.from_documents(chunks, embedder)
    retriever = vectordb.as_retriever(search_kwargs={"k": 8})

    return retriever

@st.cache_resource
def load_llm():
    return ChatOllama(model="gemma3:1b", temperature=0.1)

# Load components
retriever = load_documents()
llm = load_llm()

# Simple template
simple_template = """
You are a Customer Support Chatbot for Everstorm Outfitters. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with "I'm not sure from the docs."

CONTEXT:
{context}

USER:
{question}
"""

# Main app
st.title("🤖 Everstorm Customer Support Chatbot")
st.markdown("Ask questions about Everstorm's policies, shipping, returns, and more!")

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask a question about Everstorm..."):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Get relevant documents and generate response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            # Retrieve relevant documents
            docs = retriever.invoke(prompt)
            context = "\n\n".join([doc.page_content for doc in docs])

            # Create prompt
            formatted_prompt = simple_template.format(context=context, question=prompt)

            # Get LLM response
            response = llm.invoke(formatted_prompt)
            answer = response.content

            st.markdown(answer)

    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": answer})

# Sidebar with sample questions
with st.sidebar:
    st.header("💡 Sample Questions")
    sample_questions = [
        "What is your return policy?",
        "How long does shipping take?",
        "How can I contact customer support?",
        "What payment methods do you accept?",
        "Do you offer international shipping?"
    ]

    for question in sample_questions:
        if st.button(question, key=f"sample_{question}"):
            st.session_state.messages.append({"role": "user", "content": question})
            st.rerun()

    st.header("ℹ️ About")
    st.markdown("""
    This chatbot uses RAG (Retrieval-Augmented Generation) to answer questions about Everstorm Outfitters policies and procedures.

    **Features:**
    - 📄 PDF document processing
    - 🔍 Semantic search
    - 🤖 Local LLM (Gemma 3)
    - 💬 Conversational interface
    """)

# Footer
st.markdown("---")
st.markdown("Built with Streamlit, LangChain, and Ollama")

2025-10-18 13:31:07.266 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-18 13:31:07.267 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-18 13:31:07.268 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-10-18 13:31:07.268 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-18 13:31:07.268 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-18 13:31:07.269 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

DeltaGenerator()

Run `streamlit run app.py` from your terminal.

## 🎉 Congratulations!

You’ve just built, tested, and demoed a fully working **customer-support chatbot**.  
In one project you:

* **Prepared policy docs**: loaded and chunked them for fast retrieval.  
* **Built a vector store**: created a FAISS index with text embeddings.  
* **Plugged in an LLM**: wrapped Gemma3 with LangChain and a prompt-aware RAG chain.  
* **Validated end-to-end**: answered refund, shipping, and contact questions with confidence.  

Swap in new documents, tweak the prompt, and your store’s customers get instant, accurate answers.

👏 **Great job!** Take a moment to celebrate. The skills you used here power most RAG-based chatbots you see everywhere.
